In [1]:
from __future__ import print_function

import os
import sys
import glob
import h5py
import numpy as np
import math


import torch
from torch import nn
from torch.autograd import Variable
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.data import Dataset , DataLoader
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
from tqdm import tqdm
import tensorboard
import tensorboardX
from torch.utils.tensorboard import SummaryWriter


from log import Logger
from data import gDataset, trainDataset, testDataset
from util import r2, mse, rmse, mae, pp_mse, pp_rmse, pp_mae
from model import autoencoder_6

In [2]:
2

2

In [ ]:
if not os.path.exists('./gal_img'):
    os.mkdir('./gal_img')

    
    
def to_img(x):   # image size 56 * 56 
    x = x.view(x.size(0), 1, 56, 56)
    return x

dataset= trainDataset()
dataloader= DataLoader(dataset=dataset, batch_size=64,shuffle=True)

test_dataset = testDataset()
test_dataloader= DataLoader(dataset=test_dataset, batch_size=64,shuffle=True)


writer = SummaryWriter("exp",filename_suffix='plot_')


num_epochs =1000
batch_size = 64
learning_rate = 1e-2

model = autoencoder_6().cuda()
criterion = nn.L1Loss()

#scheduler 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[150,250,400,600,800], gamma=0.1)


for epoch in range(num_epochs):
    total_loss = 0.0
    total_mse = 0.0 
    num_examples = 0.0
    test_num_examples=0.0
    
    
    test_total_loss = 0.0    
    test_total_mse=0.0
    
    model.train()
    for data in dataloader:
        img = data
        img = img.type(torch.float32)
        img = img.view(img.size(0), 1,56,56)
        img = img.cuda()
       # print(img.shape)
        

        # forward
        output = model(img)
                
       #print("output ",output.shape)
        loss = criterion(output.cuda(), img)
        MSE_loss = nn.MSELoss()(output.cuda(), img)
        batch_size = img.size(0)
        total_loss += loss.item() * batch_size
        total_mse += MSE_loss.item() * batch_size
        num_examples += batch_size

        
        optimizer.zero_grad()
    # backward
        loss.backward()
        optimizer.step()
        scheduler.step()
        
    model.eval()
    for data in test_dataloader:
        test_img = data
        test_img = test_img.type(torch.float32)
        test_img = test_img.view(test_img.size(0), 1,56,56)
        test_img = test_img.cuda()
       # print(img.shape)
        

        # forward
        test_output = model(test_img)
                
       #print("output ",output.shape)
        test_loss = criterion(test_output.cuda(), test_img)
        test_MSE_loss = nn.MSELoss()(test_output.cuda(), test_img)
        batch_size = test_img.size(0)
        test_total_loss += test_loss.item() * batch_size
        test_total_mse += test_MSE_loss.item() * batch_size
        test_num_examples += batch_size

        
    writer.add_scalar('Loss/train',total_loss / num_examples,epoch)
    writer.add_scalar('Mse/train', total_mse / num_examples,epoch)        
    writer.add_scalar('Loss/test',test_total_loss / test_num_examples,epoch)
    writer.add_scalar('Mse/test', test_total_mse / test_num_examples,epoch)
    

        
    ''' 
    print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'
          .format(epoch + 1, num_epochs, total_loss / num_examples, total_mse/ num_examples))    
    print(' epoch [{}/{}],test_loss:{:.4f}, test_MSE_loss:{:.4f}'
          .format(epoch + 1, num_epochs, test_total_loss / test_num_examples, test_total_mse/ test_num_examples))
    '''

    if epoch % 10 == 0:
        x = to_img(img.cpu().data)
        x_hat = to_img(output.cpu().data)
        torch.save(x, './gal_img/sc_sche6_x_{}.pt'.format(epoch))
        torch.save(x_hat, './gal_img/sc_sche6_x_hat_{}.pt'.format(epoch))

torch.save(model.state_dict(), './gal_img/sc_sche6_sim_autoencoder.pth')

epoch [1/1000], loss:0.0350, MSE_loss:0.0162
 epoch [1/1000],test_loss:0.0141, test_MSE_loss:0.0018
epoch [2/1000], loss:0.0112, MSE_loss:0.0009
 epoch [2/1000],test_loss:0.0105, test_MSE_loss:0.0008
epoch [3/1000], loss:0.0102, MSE_loss:0.0007
 epoch [3/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [4/1000], loss:0.0102, MSE_loss:0.0006
 epoch [4/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [5/1000], loss:0.0102, MSE_loss:0.0006
 epoch [5/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [6/1000], loss:0.0101, MSE_loss:0.0006
 epoch [6/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [7/1000], loss:0.0101, MSE_loss:0.0006
 epoch [7/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [8/1000], loss:0.0101, MSE_loss:0.0006
 epoch [8/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [9/1000], loss:0.0101, MSE_loss:0.0006
 epoch [9/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [10/1000], loss:0.0101, MSE_loss:0.0006
 epoch [10/1000],test_loss:0.0103, test_MSE_loss:0.000

epoch [82/1000], loss:0.0101, MSE_loss:0.0006
 epoch [82/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [83/1000], loss:0.0101, MSE_loss:0.0006
 epoch [83/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [84/1000], loss:0.0101, MSE_loss:0.0006
 epoch [84/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [85/1000], loss:0.0101, MSE_loss:0.0006
 epoch [85/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [86/1000], loss:0.0101, MSE_loss:0.0006
 epoch [86/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [87/1000], loss:0.0101, MSE_loss:0.0006
 epoch [87/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [88/1000], loss:0.0101, MSE_loss:0.0006
 epoch [88/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [89/1000], loss:0.0101, MSE_loss:0.0006
 epoch [89/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [90/1000], loss:0.0101, MSE_loss:0.0006
 epoch [90/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [91/1000], loss:0.0101, MSE_loss:0.0006
 epoch [91/1000],test_loss:0.0103, t

epoch [162/1000], loss:0.0101, MSE_loss:0.0006
 epoch [162/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [163/1000], loss:0.0101, MSE_loss:0.0006
 epoch [163/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [164/1000], loss:0.0101, MSE_loss:0.0006
 epoch [164/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [165/1000], loss:0.0101, MSE_loss:0.0006
 epoch [165/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [166/1000], loss:0.0101, MSE_loss:0.0006
 epoch [166/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [167/1000], loss:0.0101, MSE_loss:0.0006
 epoch [167/1000],test_loss:0.0103, test_MSE_loss:0.0008
epoch [168/1000], loss:0.0101, MSE_loss:0.0006
 epoch [168/1000],test_loss:0.0103, test_MSE_loss:0.0008


In [ ]:
torch.ones(1,1).shape

In [ ]:

img.size()

In [ ]:
a = loss.cpu().detach().numpy()
a


In [ ]:
a

In [ ]:
vis = visdom.Visdom()


loss_window = vis.line(
    X=torch.zeros((1,)).cpu(),
    Y=torch.zeros((1)).cpu(),
    opts=dict(xlabel='epoch',ylabel='Loss',title='training loss',legend=['Loss']))

vis.line(X=torch.ones((1,1)).cpu()*epoch,Y=torch.Tensor([loss.data]).unsqueeze(0).cpu(),win=loss_window,update='append')


In [ ]:
class autoencoder_5(nn.Module):
    def __init__(self):
        super(autoencoder_5, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1),  
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2), 
            
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1),  
            
            nn.Conv2d(8, 2, 3, stride=1, padding=1),  # b, 2,
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1),  # b, 2,
            
            nn.Conv2d(2, 1, 3, stride=2, padding=1),  # b, 1, 3, 3 
            nn.ReLU(True)
        )
        self.decoder = nn.Sequential(
            
            # output = (input - 1 ) * stride -2 * padding + kernel size
            
            nn.ConvTranspose2d(1, 2, 3, stride=1),  # b, 2,  4 , 4 
            nn.ReLU(True),
            
            nn.ConvTranspose2d(2, 8, 3, stride=3, padding =1  ),  # b, 8, 10 , 10 
            nn.ReLU(True),
            
            nn.ConvTranspose2d(16, 8, 3, stride=2, padding =1  ),  # b, 8, 19 , 19 
            nn.ReLU(True),

            nn.ConvTranspose2d(8, 4, 3, stride=3, padding =1  ),  # b, 8,  55, 55  
            nn.ReLU(True),

            nn.ConvTranspose2d(4, 1, 3, stride=3, padding =1  ),  # b, 8,  55, 55  
            nn.Sigmoid()


        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out
    
    
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)

        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out
class Encoder(nn.Module):

    def __init__(self, block, layers, num_classes=23):
        self.inplanes = 64
        super (Encoder, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)#, return_indices = True)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, 1000)
	#self.fc = nn.Linear(num_classes,16) 
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
	
        x = self.bn1(x)
        x = self.relu(x)
	
        x = self.maxpool(x)
	
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x
    
encoder = Encoder(Bottleneck, [3, 4, 6, 3])

encoder.fc = nn.Linear(2048, 48)

encoder=encoder.cuda()
y=torch.rand(1,3,224,224)
x=torch.rand(1,128)
x=Variable(x.cuda())

zsize=10  
    
class Decoder(nn.Module):
	def __init__(self):
		super(Decoder,self).__init__()
		self.dfc3 = nn.Linear(zsize, 4096)
		self.bn3 = nn.BatchNorm2d(4096)
		self.dfc2 = nn.Linear(4096, 4096)
		self.bn2 = nn.BatchNorm2d(4096)
		self.dfc1 = nn.Linear(4096,256 * 6 * 6)
		self.bn1 = nn.BatchNorm2d(256*6*6)
		self.upsample1=nn.Upsample(scale_factor=2)
		self.dconv5 = nn.ConvTranspose2d(256, 256, 3, padding = 0)
		self.dconv4 = nn.ConvTranspose2d(256, 384, 3, padding = 1)
		self.dconv3 = nn.ConvTranspose2d(384, 192, 3, padding = 1)
		self.dconv2 = nn.ConvTranspose2d(192, 64, 5, padding = 2)
		self.dconv1 = nn.ConvTranspose2d(64, 3, 12, stride = 4, padding = 4)

	def forward(self,x):#,i1,i2,i3):
		
		x = self.dfc3(x)
		#x = F.relu(x)
		x = F.relu(self.bn3(x))
		
		x = self.dfc2(x)
		x = F.relu(self.bn2(x))
		#x = F.relu(x)
		x = self.dfc1(x)
		x = F.relu(self.bn1(x))
		#x = F.relu(x)
		#print(x.size())
		x = x.view(batch_size,256,6,6)
		#print (x.size())
		x=self.upsample1(x)
		#print x.size()
		x = self.dconv5(x)
		#print x.size()
		x = F.relu(x)
		#print x.size()
		x = F.relu(self.dconv4(x))
		#print x.size()
		x = F.relu(self.dconv3(x))
		#print x.size()		
		x=self.upsample1(x)
		#print x.size()		
		x = self.dconv2(x)
		#print x.size()		
		x = F.relu(x)
		x=self.upsample1(x)
		#print x.size()
		x = self.dconv1(x)
		#print x.size()
		x = F.sigmoid(x)
		#print x
		return x
decoder = Decoder()




class autoencoder(nn.Module):
	def __init__(self):
		super(autoencoder,self).__init__()
		self.encoder = encoder(x)
		self.decoder = decoder()

	def forward(self,x):
		#x=Encoder(x)
		x = self.encoder(x)

		x = self.decoder(x)
		return x
    
    
    
    
    
 

In [ ]:




def plot_sample_img(img, name):
    img = img.view(1, 56, 56)
    save_image(img, './sample_{}.png'.format(name))


def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor


def tensor_round(tensor):
    return torch.round(tensor)